In [1]:
%matplotlib inline
import importlib
import utils2; importlib.reload(utils2)
from utils2 import *

from scipy.optimize import fmin_l_bfgs_b
from imageio import imwrite
from keras import metrics

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path = '../datasets/super-res/'

In [3]:
import bcolz
arr_lr = bcolz.open(path + 'trn_resized_72.bc')[:]
arr_hr = bcolz.open(path + 'trn_resized_288.bc')[:]

In [4]:
parms = {'verbose': 0, 'callbacks': [TQDMNotebookCallback(leave_inner=True)]}

In [5]:
def conv_block(x, nfilters, size, strides=(2, 2), mode='same', act=True):
    x = Conv2D(nfilters, size, strides=strides, padding=mode)(x)
    x = BatchNormalization()(x)
    return Activation('relu')(x) if act else x

In [6]:
??Conv2D

In [7]:
??BatchNormalization

In [8]:
def res_block(ip, nfilters=64):
    x = conv_block(ip, nfilters, 3, (1, 1))
    x = conv_block(x, nfilters, 3, (1, 1), act=False)
    return merge([x, ip], mode='sum')

In [9]:
def deconv_block(x, nfilters, size, strides=(2, 2)):
    x = Deconv2D(nfilters, size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    return Activation('relu')(x)

In [10]:
??Deconv2D

In [11]:
def up_block(x, nfilters, size):
    x = keras.layers.UpSampling2D()(x)
    x = Conv2D(x, nfilters, size, padding='same')(x)
    x = BatchNormalization()(x)
    return Activation('relu')(x)

In [12]:
??keras.layers.UpSampling2D

In [13]:
??Input

In [14]:
inp = Input(shape=arr_lr.shape[1:])
x = conv_block(inp, 64, 9, (1, 1))
for i in range(4): 
    x = res_block(x)
x = deconv_block(x, 64, 3)
x = deconv_block(x, 64, 3)
x = Conv2D(3, (9, 9), activation='tanh', padding='same')(x)
outp = Lambda(lambda x: (x + 1) * 127.5)(x)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [15]:
outp

<tf.Tensor 'lambda_1/mul:0' shape=(?, ?, ?, 3) dtype=float32>

In [16]:
shp = arr_hr.shape[1:]
shp

(288, 288, 3)

In [17]:
rn_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32)
preproc = lambda x: (x - rn_mean)[:, :, :, ::-1]
deproc = lambda x,s: np.clip(x.reshape(s)[:, :, :, ::-1] + rn_mean, 0, 255)

In [18]:
vgg_inp = Input(shp)
vgg = VGG16(include_top=False, input_tensor=Lambda(preproc)(vgg_inp))

In [19]:
for l in vgg.layers:
    l.trainable=False

In [20]:
def get_vgg_outp(ln): return vgg.get_layer('block{}_conv1'.format(ln)).output
vgg_content = Model(vgg_inp, [get_vgg_outp(l) for l in [1, 2, 3]])
